In [1]:
import yfinance as yf
import re
import pandas as pd
from pandas import DataFrame, Series

In [2]:
%pylab inline
from datetime import datetime
#from yahoofinancials import YahooFinancials

Populating the interactive namespace from numpy and matplotlib


In [3]:
SP = yf.Ticker('ES=F')

In [7]:
df = SP.history(period = '7d', interval = '1m')

In [8]:
df.shape

(7669, 7)

In [9]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2021-08-27 00:00:00-04:00,4475.25,4475.25,4475.25,4475.25,0,0,0
2021-08-27 00:01:00-04:00,4475.25,4475.25,4475.25,4475.25,10,0,0
2021-08-27 00:02:00-04:00,4475.25,4475.25,4475.25,4475.25,38,0,0
2021-08-27 00:03:00-04:00,4475.25,4475.25,4475.25,4475.25,7,0,0
2021-08-27 00:04:00-04:00,4475.25,4475.25,4475.25,4475.25,14,0,0
...,...,...,...,...,...,...,...
2021-09-03 15:32:00-04:00,4538.00,4538.00,4537.50,4537.75,1481,0,0
2021-09-03 15:33:00-04:00,4537.75,4538.50,4537.50,4538.25,1104,0,0
2021-09-03 15:34:00-04:00,4538.25,4538.75,4538.00,4538.75,929,0,0


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.layers import LeakyReLU

In [11]:
RNN_df = df[['Close', 'Volume']]
RNN_df

,Close,Volume
Datetime,,
2021-08-27 00:00:00-04:00,4475.25,0
2021-08-27 00:01:00-04:00,4475.25,10
2021-08-27 00:02:00-04:00,4475.25,38
2021-08-27 00:03:00-04:00,4475.25,7
2021-08-27 00:04:00-04:00,4475.25,14
...,...,...
2021-09-03 15:32:00-04:00,4537.75,1481
2021-09-03 15:33:00-04:00,4538.25,1104
2021-09-03 15:34:00-04:00,4538.75,929


In [12]:
#notes for prediction
seq_len = 60
future_period_predict = 2
ratio_to_predict= 'ES' 

#classification target
def classify(current, dollar_change):
    tot_change=0
    for x in RNN_df['dollar_change']:
        tot_change = tot_change + x
        if dollar_change >= 1:
            return 1
        elif dollar_change <= -0.5:
            return 0 
        elif tot_change<0:
            tot_change=0
            continue
        else:
            continue
#shift timeseries down 3 spots for 3 hour prediction
dollar_change = ((RNN_df['Close'].shift(-future_period_predict))-RNN_df['Close'])

#create a 3 hour future column
RNN_df["dollar_change"] = dollar_change
RNN_df['Future'] = RNN_df['Close'].shift(-future_period_predict)
#RNN_df['Future double'] = RNN_df['Close'].shift(-future_period_predict*5)
RNN_df

<ipython-input-12-86e718e3b9a2>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RNN_df["dollar_change"] = dollar_change
<ipython-input-12-86e718e3b9a2>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RNN_df['Future'] = RNN_df['Close'].shift(-future_period_predict)


,Close,Volume,dollar_change,Future
Datetime,,,,
2021-08-27 00:00:00-04:00,4475.25,0,0.00,4475.25
2021-08-27 00:01:00-04:00,4475.25,10,0.00,4475.25
2021-08-27 00:02:00-04:00,4475.25,38,0.00,4475.25
2021-08-27 00:03:00-04:00,4475.25,7,0.00,4475.25
2021-08-27 00:04:00-04:00,4475.25,14,0.00,4475.25
...,...,...,...,...
2021-09-03 15:32:00-04:00,4537.75,1481,1.00,4538.75
2021-09-03 15:33:00-04:00,4538.25,1104,0.25,4538.50
2021-09-03 15:34:00-04:00,4538.75,929,-0.50,4538.25


In [13]:
RNN_df=RNN_df.dropna()

#send close and 3d future to classify function
def grab_arguments(s):
    return classify(s[0],s[1])

#create a classification
RNN_df['target'] = RNN_df[['Close','dollar_change']].T.apply(grab_arguments)


<ipython-input-13-ee1eb99ad8d8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RNN_df['target'] = RNN_df[['Close','dollar_change']].T.apply(grab_arguments)


In [14]:
RNN_df



,Close,Volume,dollar_change,Future,target
Datetime,,,,,
2021-08-27 00:00:00-04:00,4475.25,0,0.00,4475.25,NaN
2021-08-27 00:01:00-04:00,4475.25,10,0.00,4475.25,NaN
2021-08-27 00:02:00-04:00,4475.25,38,0.00,4475.25,NaN
2021-08-27 00:03:00-04:00,4475.25,7,0.00,4475.25,NaN
2021-08-27 00:04:00-04:00,4475.25,14,0.00,4475.25,NaN
...,...,...,...,...,...
2021-09-03 15:30:00-04:00,4536.75,1346,1.00,4537.75,1.0
2021-09-03 15:31:00-04:00,4537.75,2189,0.50,4538.25,NaN
2021-09-03 15:32:00-04:00,4537.75,1481,1.00,4538.75,1.0


In [15]:

#mask= (RNN_df['target'] == 0) or (RNN_df['target'] == 1)
RNN_df = RNN_df.fillna(0)
RNN_df = RNN_df.T.drop(['Future']).T
RNN_df = RNN_df.T.drop(['dollar_change']).T
#train_set = train_set.T.drop(['Future']).T
#test_set = test_set.T.drop(['Future']).T
#train_set = train_set.T.drop(['dollar_change']).T
#test_set = test_set.T.drop(['dollar_change']).T
RNN_df

,Close,Volume,target
Datetime,,,
2021-08-27 00:00:00-04:00,4475.25,0.0,0.0
2021-08-27 00:01:00-04:00,4475.25,10.0,0.0
2021-08-27 00:02:00-04:00,4475.25,38.0,0.0
2021-08-27 00:03:00-04:00,4475.25,7.0,0.0
2021-08-27 00:04:00-04:00,4475.25,14.0,0.0
...,...,...,...
2021-09-03 15:30:00-04:00,4536.75,1346.0,1.0
2021-09-03 15:31:00-04:00,4537.75,2189.0,0.0
2021-09-03 15:32:00-04:00,4537.75,1481.0,1.0


In [16]:
################################
##split into train and test set
#############################

#make sure index is sorted. probs already was
times = sorted(RNN_df.index)

# last 5% of time as test set
last_10pct = times[-int(0.1*len(times))]
print(last_10pct)


2021-09-03 02:29:00-04:00


In [17]:
# test set is last 10% and beyond
test_set = RNN_df[RNN_df.index >= last_10pct]
test_set
#train set is anything under the last 10%
train_set =RNN_df[(RNN_df.index < last_10pct)]
train_set

,Close,Volume,target
Datetime,,,
2021-08-27 00:00:00-04:00,4475.25,0.0,0.0
2021-08-27 00:01:00-04:00,4475.25,10.0,0.0
2021-08-27 00:02:00-04:00,4475.25,38.0,0.0
2021-08-27 00:03:00-04:00,4475.25,7.0,0.0
2021-08-27 00:04:00-04:00,4475.25,14.0,0.0
...,...,...,...
2021-09-03 02:24:00-04:00,4542.50,133.0,0.0
2021-09-03 02:25:00-04:00,4542.50,32.0,0.0
2021-09-03 02:26:00-04:00,4542.25,111.0,0.0


In [18]:
#Drop the right answer, 3 hour future
#train_set = train_set.T.drop(['Future']).T
#test_set = test_set.T.drop(['Future']).T
#train_set = train_set.T.drop(['dollar_change']).T
#test_set = test_set.T.drop(['dollar_change']).T
len(train_set)
len(test_set)

766

In [19]:
#test_set

In [20]:
## check for infinte values in your data

#train_set['Volume'].preprocessing.scale(train_set['Volume'])
import numpy as np
#check for inf as well as -inf
#mask1 = isinf(-train_set['Volume'])
#train_set[mask1]


#mask = isinf(test_set['Volume'])
#test_set[mask]

# mask = train_set['Volume'] == float(inf)
# train_set[mask]= 0

train_set.loc[(train_set['Volume'] == float("inf")),'Volume']=0
train_set[train_set["Volume"] == 0]

test_set.loc[(test_set['Volume'] == float("inf")),'Volume']=0
test_set[test_set["Volume"] == 0]

# Replacing infinite with nan
#train_set.replace([np.inf, -np.inf], np.nan, inplace=True)
  
# Dropping all the rows with nan values
#train_set.dropna(inplace=True)
  
# Printing df


# Replacing infinite with nan
#test_set.replace([np.inf, -np.inf], np.nan, inplace=True)
  
# Dropping all the rows with nan values
#test_set.dropna(inplace=True)
  
# Printing df
#len(test_set)
#len(train_set)
#find the infinite values to get rid of
# 2021-05-05 13:30:00-04:00
#train_set.T["2021-05-05 13:30:00-04:00"]
#drop that index or indices
#RNN_df=RNN_df[~RNN_df.isin([np.nan, np.inf, -np.inf]).any(1)]
#len(RNN_df)

C:\Users\chand\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\chand\Anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,Close,Volume,target
Datetime,,,


In [21]:
mask = (test_set['Volume'] == float("inf"))
test_set[mask]

,Close,Volume,target
Datetime,,,


In [22]:
#train_set = train_set.drop('2019-12-02 09:30:00-05:00')
#train_set = train_set.drop('2019-12-26 09:30:00-05:00')
#train_set = train_set.drop('2020-05-14 12:30:00-04:00')
#train_set = train_set.drop('2020-12-28 09:30:00-05:00')
#train_set = train_set.drop('2021-05-05 14:30:00-04:00')
#train_set = train_set.drop('2020-11-30 09:30:00-05:00')

In [23]:
###################
#Preprocessing - scale and normalize
#####################

#import preprocessing
from sklearn import preprocessing

# import deque continual appending to list and maintains list len
from collections import deque

import numpy as np
import random

def preprocess_df(df):
    df.dropna(inplace=True)
    
    for col in df:
        
        if col != "target":
            #normalize with % change 
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            #scale the data
            df.loc[(df['Volume'] == float("inf")),'Volume']=0
            #print(df[df[col] == float("inf")])
            df[col]= preprocessing.scale(df[col].values)
        
    
    df.dropna(inplace = True)
    
    sequential_data = []
    #maintain the seq_len
    prev_days = deque(maxlen=seq_len)
    #turns df into a list of lists
    for i in df.values:
        # take each value except for target col in list
        prev_days.append([n for n in i[:-1]])
        #print(len(prev_days))
        if len(prev_days) == seq_len:
            sequential_data.append([np.array(prev_days), i[-1]])
            #print(sequential_data)
    
    random.shuffle(sequential_data)  
    
    #####################
    ### Balancet the Data 
    ######################

    buys = []
    sells = []
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq,target])
        elif target ==1:
            buys.append([seq,target])
     
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys), len(sells)) 
    buys = buys[:lower]
    sells = sells[:lower]
    #print(sells)
    sequential_data = buys+sells
    
    random.shuffle(sequential_data)
    
    x = []
    y = []
    
    for seq, target in sequential_data:
        x.append(seq)
        y.append(target)
        
    return np.array(x), y 

#preprocess_df(train_set)
train_x, train_y = preprocess_df(train_set)
test_x, test_y = preprocess_df(test_set)

C:\Users\chand\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
<ipython-input-23-06c1fe99da72>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].pct_change()
C:\Users\chand\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [24]:
RNN_df

,Close,Volume,target
Datetime,,,
2021-08-27 00:00:00-04:00,4475.25,0.0,0.0
2021-08-27 00:01:00-04:00,4475.25,10.0,0.0
2021-08-27 00:02:00-04:00,4475.25,38.0,0.0
2021-08-27 00:03:00-04:00,4475.25,7.0,0.0
2021-08-27 00:04:00-04:00,4475.25,14.0,0.0
...,...,...,...
2021-09-03 15:30:00-04:00,4536.75,1346.0,1.0
2021-09-03 15:31:00-04:00,4537.75,2189.0,0.0
2021-09-03 15:32:00-04:00,4537.75,1481.0,1.0


In [25]:
print(f"train data:{len(train_x)} validation: {len(test_x)}")

train data:1112 validation: 166


In [26]:
print(f"Dont buys:{train_y.count(0)}, buys:{train_y.count(1)}")

Dont buys:556, buys:556


In [27]:
print(f"VALIDATION Dont buys: {test_y.count(0)}, buys: {test_y.count(1)}")

VALIDATION Dont buys: 83, buys: 83


In [28]:

epochs = 15
batch_size = 64
name = f"{seq_len}-SEQ-{future_period_predict}-PRED-{int(time.time())}"



In [29]:
# check to make sure training and test set are numpy arrays
test_y = np.asarray(test_y)
train_y = np.asarray(train_y)

In [30]:
model = Sequential()
model.add(LSTM(128, input_shape= (train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape= (train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape= (train_x.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(tf.keras.layers.Flatten())
#dense layer activation rectified linear


model.add(Dense(32, activation ='relu'))
model.add(Dropout(0.2))

#output layer
model.add(Dense(2, activation = 'softmax'))

opt = tf.keras.optimizers.SGD(lr= 0.0002, decay=1e-3)
model.compile(loss='sparse_categorical_crossentropy',
             optimizer = opt,
             metrics= ['accuracy'])

tensorboard = TensorBoard(log_dir = f'log/{name}')


checkpoint = ModelCheckpoint(filepath= r'C:\Users\chand\My Python\models',verbose=1, save_best_only=True)

history = model.fit(
            train_x, train_y,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(test_x, test_y),
            callbacks=[tensorboard, checkpoint])
#model.save('')

Epoch 1/15
 1/18 [>.............................] - ETA: 0s - loss: 0.9679 - accuracy: 0.4844WARNING:tensorflow:From C:\Users\chand\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
18/18 [==============================] - ETA: 0s - loss: 0.9058 - accuracy: 0.4910
Epoch 00001: val_loss improved from inf to 0.69322, saving model to C:\Users\chand\My Python\models
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\chand\My Python\models\assets
18/18 [==============================] - 81s 5s/step - loss: 0.9058 - accuracy: 0.4910 - val_loss: 0.6932 - val_accuracy: 0.5482
Epoch